In [2]:
import folium
import pandas as pd
from folium.plugins import HeatMapWithTime
from datetime import datetime, timedelta

def create_taxi_heatmap(df):
    # Convert pickup times to datetime
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    
    # Group data by pickup zone and minute
    grouped_data = df.groupby([
        'pickup_zone', 
        df['tpep_pickup_datetime'].dt.strftime('%H:%M')
    ]).size().reset_index(name='count')
    
    grouped_data.rename(columns={'tpep_pickup_datetime': 'minute'}, inplace=True)
    
    # Merge with unique pickup zone coordinates
    grouped_data = pd.merge(
        grouped_data, 
        df[['pickup_zone', 'pickup_lat', 'pickup_lon']].drop_duplicates(),
        on='pickup_zone', 
        how='left'
    )
    
    # Prepare heatmap data for each minute
    all_minutes = sorted(df['tpep_pickup_datetime'].dt.strftime('%H:%M').unique())
    minute_data = []
    for minute in all_minutes:
        minute_group = grouped_data[grouped_data['minute'] == minute]
        locations = [[row['pickup_lat'], row['pickup_lon'], row['count']] 
                    for _, row in minute_group.iterrows()]
        minute_data.append(locations)
    
    nyc_map = folium.Map(
        location=[40.7528, -74.0060],
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    HeatMapWithTime(
        minute_data,
        index=all_minutes,
        auto_play=True,
        max_opacity=0.8,
        radius=15,
        min_opacity=0.3,
        use_local_extrema=True,
        display_index=True,
    ).add_to(nyc_map)
    
    title_html = '''
        <div style="position: fixed; 
                    top: 10px; 
                    left: 50px; 
                    z-index:9999; 
                    background-color: white;
                    border: 2px solid black;
                    border-radius: 5px;
                    padding: 10px;
                    font-size: 16px;
                    font-weight: bold;">
            NYC Taxi Pickups by Minute
        </div>
    '''
    nyc_map.get_root().html.add_child(folium.Element(title_html))
    
    return nyc_map

df = pd.read_csv("../../data/taxi_data.csv")
map_with_slider = create_taxi_heatmap(df)
map_with_slider.save('collaborative.html')